<a href="https://www.kaggle.com/code/oladitisamuel/notebook22e5a53af4?scriptVersionId=212817104" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
import warnings as w
w.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

# Load the dataset
dataset = pd.read_csv('/kaggle/input/global-ev-data/Global EV Data 2024.csv')

# Create a copy of the dataset
df = dataset.copy()

# Display the first few rows
print(df.head())


In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

In [ ]:
import pandas as pd
import numpy as np  # Ensure numpy is imported for numeric checks

def colm(df):
    # Categorical columns: category, object, or bool
    cat_cols = [col for col in df.columns if df[col].dtypes in ['category', 'object', 'bool']]
    
    # Numeric but categorical: less than 10 unique values
    num_but_cat = [col for col in df.columns if df[col].nunique(dropna=True) < 10 and pd.api.types.is_numeric_dtype(df[col])]
    
    # Cardinal columns: Categorical with more than 15 unique values
    cat_but_car = [col for col in df.columns if df[col].nunique(dropna=True) > 15 and df[col].dtypes in ['category', 'object']]
    
    # Combine and filter categorical columns
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    
    # Numeric columns: Numeric types excluding the ones classified as categorical
    num_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
    num_cols = [col for col in num_cols if col not in cat_cols]
    
    # Print categorized columns
    print(f'Categorical Columns: {cat_cols} \nNumeric Columns: {num_cols} \nCardinal Columns: {cat_but_car}')
    
    return cat_cols, num_cols, cat_but_car


In [ ]:
catC,numC,catdC = colm(df)

In [ ]:
import matplotlib.pyplot as plt

# Create a 3x2 grid of subplots with adjusted size
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(12, 15))

# Plot pie charts
df['mode'].value_counts().plot(kind='pie', ax=ax1, autopct='%1.1f%%', startangle=90)
ax1.set_title('Mode Frequency')
ax1.set_ylabel('')  # Remove y-axis label for pie chart

df['powertrain'].value_counts().plot(kind='pie', ax=ax2, autopct='%1.1f%%', startangle=90)
ax2.set_title('Powertrain Frequency')
ax2.set_ylabel('')

df['parameter'].value_counts().plot(kind='pie', ax=ax3, autopct='%1.1f%%', startangle=90)
ax3.set_title('Parameter Frequency')
ax3.set_ylabel('')

df['category'].value_counts().plot(kind='pie', ax=ax4, autopct='%1.1f%%', startangle=90)
ax4.set_title('Category Frequency')
ax4.set_ylabel('')

df['unit'].value_counts().plot(kind='pie', ax=ax5, autopct='%1.1f%%', startangle=90)
ax5.set_title('Unit Frequency')
ax5.set_ylabel('')

# Plot bar chart for region
rc = df['region'].value_counts()
rcf = rc[rc > 350]  # Filter regions with counts > 350

if not rcf.empty:  # Ensure there is data to plot
    rcf.plot(kind='bar', ax=ax6, color='skyblue', edgecolor='black')
    ax6.set_title('Region Frequency (>350)')
    ax6.set_ylabel('Frequency')
    ax6.set_xlabel('Region')
else:
    ax6.set_visible(False)  # Hide the subplot if no data meets the condition

# Adjust layout
plt.tight_layout()
plt.show()

# Save the figure as a PNG file
plt.savefig('output.png', dpi=300, bbox_inches='tight')  # Adjust DPI and layout
plt.show()

In [ ]:
import plotly.express as px

# Group by 'year' and calculate the mean of 'value'
value_gb = df.groupby('year')['value'].mean().reset_index()

# Plot using Plotly Express
fig = px.line(value_gb, x='year', y='value', title='Yearly Average Values')
fig.show()


In [ ]:
import plotly.express as px

# Group by 'parameter', calculate the mean of 'value', and sort in descending order
value_gb_pt = df.groupby('parameter')['value'].mean().sort_values(ascending=False).reset_index()

# Plot using Plotly Express
fig = px.bar(value_gb_pt, x='parameter', y='value', title='Average Value by Parameter')
fig.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the figure size
plt.figure(figsize=(12, 6))

# Create the line plot
sns.lineplot(data=df, x='year', y='value', hue='powertrain')

# Set plot title and labels
plt.title('Value Change by Year and Powertrain Type')
plt.xlabel('Year')
plt.ylabel('Value')

# Set the y-axis to log scale (ensure that values are positive)
plt.yscale('log')

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Set the figure size
plt.figure(figsize=(12, 6))

# Get the top 8 regions with the highest average value
top_10_regions = df.groupby('region')['value'].mean().sort_values(ascending=False).head(8)

# Plot the bar chart
top_10_regions.plot(kind='bar')

# Set the plot title and labels in English
plt.title('Top 8 Regions with the Highest Average Value')
plt.xlabel('Region')
plt.ylabel('Average Value')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.show()


# **Tesla Stock Market**

In [ ]:
!pip install chart-studio


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 

In [ ]:
TSLA = pd.read_csv('/kaggle/input/tesla-stock-market/Tesla_raw_data.csv')

TSLA.head()


In [ ]:
print(TSLA.columns)

In [ ]:
TSLA['volume'].describe()

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Check the first few rows of the dataset to understand its structure
print(TSLA.head())

# Check for missing values and data types
print(TSLA.info())

# Check summary statistics of the dataset
print(TSLA.describe())

# Clean the dataset if needed (for example, drop rows with missing values)
TSLAdf = TSLA.dropna()  # Or handle missing values as required

# Convert date column to datetime format (if it's not already)
TSLAdf['date'] = pd.to_datetime(TSLAdf['date'])

# Set the date column as the index (for time series analysis)
TSLAdf.set_index('date', inplace=True)

# Basic analysis: Average closing price
avg_close_price = TSLAdf['close'].mean()
print(f'Average closing price: {avg_close_price}')



In [ ]:
# Visualization 1: Plot the stock price (closing price) over time
plt.figure(figsize=(14, 7))
plt.plot(TSLAdf.index, TSLAdf['close'], label='Closing Price', color='b')
plt.title('Tesla Stock Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
print(TSLAdf.columns)


In [ ]:

# Visualization 2: Plot the stock price (adjusted closing price) over time
plt.figure(figsize=(14, 7))
plt.plot(TSLAdf.index, TSLAdf['adjusted_close'], label='Adjusted Closing Price', color='g')
plt.title('Tesla Adjusted Stock Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
# Visualization 3: Distribution of daily returns (percentage change)
TSLAdf['Daily Return'] = TSLAdf['close'].pct_change() * 100  # Calculate daily returns as percentage change

plt.figure(figsize=(12, 6))
sns.histplot(TSLAdf['Daily Return'].dropna(), bins=50, kde=True, color='purple')
plt.title('Distribution of Tesla Daily Returns')
plt.xlabel('Daily Return (%)')
plt.ylabel('Frequency')
plt.show()



In [ ]:
# Correlation Matrix for Stock Data (you can compare open, close, high, low, volume)
corr_matrix = TSLAdf[['open', 'high', 'low', 'close', 'adjusted_close', 'volume']].corr()

# Heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Tesla Stock Data')
plt.show()



In [ ]:
# Optional: Calculate rolling averages (e.g., 30-day moving average)
TSLAdf['30_day_MA'] = TSLAdf['close'].rolling(window=30).mean()  # Calculate 30-day moving average

# Visualization 4: Plot closing price and 30-day moving average
plt.figure(figsize=(14, 7))
plt.plot(TSLAdf.index, TSLAdf['close'], label='Closing Price', color='blue')
plt.plot(TSLAdf.index, TSLAdf['30_day_MA'], label='30-Day Moving Average', color='red', linestyle='--')
plt.title('Tesla Stock Price and 30-Day Moving Average')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import plotly.graph_objs as go
from plotly.offline import iplot
import pandas as pd

# Assuming TSLA is your DataFrame and 'date' and 'close' are the correct column names
TSLA['date'] = pd.to_datetime(TSLA['date'])  # Ensure 'date' is in datetime format

# Create a scatter plot trace for the Tesla stock data
trace = go.Scatter(
    x=TSLA['date'],  # x-axis is 'date'
    y=TSLA['close'],  # y-axis is 'close'
    mode='lines',  # 'lines' mode to connect the points with a line
    name='Tesla Stock Price'
)

# Define the layout for the plot
layout = go.Layout(
    title='Stock Prices of TESLA',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickformat="%Y-%m-%d"  # Optional: Format the date for better readability
    ),
    yaxis=dict(
        title='Price',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

# Create the plot figure
plot = go.Figure(data=[trace], layout=layout)

# Display the plot inline (for Jupyter notebooks)
iplot(plot)


In [ ]:
!pip install dash


In [ ]:
from dash import Dash, dcc, html
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np

# Assuming TSLA DataFrame is already prepared
TSLA['date'] = pd.to_datetime(TSLA['date'])
TSLA['Daily Return'] = TSLA['close'].pct_change() * 100
TSLA['30_day_MA'] = TSLA['close'].rolling(window=30).mean()

# Initialize Dash app
app = Dash(__name__)

# Visualization 1: Tesla Stock Price (Closing Price)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['close'], mode='lines', name='Closing Price'))
fig1.update_layout(title='Tesla Stock Prices (Closing Price)', xaxis_title='Date', yaxis_title='Price (USD)')

# Visualization 2: Tesla Adjusted Stock Price
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['adjusted_close'], mode='lines', name='Adjusted Closing Price', line=dict(color='green')))
fig2.update_layout(title='Tesla Adjusted Stock Prices', xaxis_title='Date', yaxis_title='Price (USD)')

# Visualization 3: Distribution of Daily Returns
fig3 = px.histogram(TSLA, x='Daily Return', nbins=50, title='Distribution of Tesla Daily Returns', color_discrete_sequence=['purple'])
fig3.update_layout(xaxis_title='Daily Return (%)', yaxis_title='Frequency')

# Visualization 4: Correlation Matrix
corr_matrix = TSLA[['open', 'high', 'low', 'close', 'adjusted_close', 'volume']].corr()
fig4 = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu',  # Use a valid Plotly colorscale
    title='Correlation Matrix of Tesla Stock Data'
)

# Visualization 5: Closing Price with 30-Day Moving Average
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['close'], mode='lines', name='Closing Price'))
fig5.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['30_day_MA'], mode='lines', name='30-Day Moving Average', line=dict(dash='dash', color='red')))
fig5.update_layout(title='Tesla Stock Prices with 30-Day Moving Average', xaxis_title='Date', yaxis_title='Price (USD)')

# App layout
app.layout = html.Div([
    html.H1("Tesla Stock Dashboard", style={'textAlign': 'center'}),
    html.Div([
        dcc.Graph(figure=fig1, style={'width': '48%', 'display': 'inline-block'}),
        dcc.Graph(figure=fig2, style={'width': '48%', 'display': 'inline-block'}),
    ]),
    html.Div([
        dcc.Graph(figure=fig3, style={'width': '48%', 'display': 'inline-block'}),
        dcc.Graph(figure=fig4, style={'width': '48%', 'display': 'inline-block'}),
    ]),
    html.Div([
        dcc.Graph(figure=fig5, style={'width': '98%', 'margin': '0 auto'}),
    ])
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



In [ ]:
!pip install plotly fpdf
!pip install -U kaleido


In [ ]:
pip install -U kaleido


In [ ]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
from plotly.io import write_image
from fpdf import FPDF

# Assuming TSLA DataFrame is already prepared
TSLA['date'] = pd.to_datetime(TSLA['date'])
TSLA['Daily Return'] = TSLA['close'].pct_change() * 100
TSLA['30_day_MA'] = TSLA['close'].rolling(window=30).mean()

# Visualization 1: Tesla Stock Price (Closing Price)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['close'], mode='lines', name='Closing Price'))
fig1.update_layout(title='Tesla Stock Prices (Closing Price)', xaxis_title='Date', yaxis_title='Price (USD)')
fig1.write_image("figure1.png")

# Visualization 2: Tesla Adjusted Stock Price
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['adjusted_close'], mode='lines', name='Adjusted Closing Price', line=dict(color='green')))
fig2.update_layout(title='Tesla Adjusted Stock Prices', xaxis_title='Date', yaxis_title='Price (USD)')
fig2.write_image("figure2.png")

# Visualization 3: Distribution of Daily Returns
fig3 = px.histogram(TSLA, x='Daily Return', nbins=50, title='Distribution of Tesla Daily Returns', color_discrete_sequence=['purple'])
fig3.update_layout(xaxis_title='Daily Return (%)', yaxis_title='Frequency')
fig3.write_image("figure3.png")

# Visualization 4: Correlation Matrix
corr_matrix = TSLA[['open', 'high', 'low', 'close', 'adjusted_close', 'volume']].corr()
fig4 = px.imshow(corr_matrix, text_auto=True, color_continuous_scale='RdBu', title='Correlation Matrix of Tesla Stock Data')
fig4.write_image("figure4.png")

# Visualization 5: Closing Price with 30-Day Moving Average
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['close'], mode='lines', name='Closing Price'))
fig5.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['30_day_MA'], mode='lines', name='30-Day Moving Average', line=dict(dash='dash', color='red')))
fig5.update_layout(title='Tesla Stock Prices with 30-Day Moving Average', xaxis_title='Date', yaxis_title='Price (USD)')
fig5.write_image("figure5.png")

# Combine images into a PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Add visualizations to PDF
for i in range(1, 6):
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f"Visualization {i}", ln=True, align='C')
    pdf.image(f"figure{i}.png", x=10, y=20, w=180)

# Save the PDF
pdf.output("Tesla_Stock_Dashboard.pdf")

print("PDF generated successfully!")


In [ ]:
from dash import dcc, html
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

# Simulate Tesla data (replace with your dataset)
TSLA = pd.DataFrame({
    'date': pd.date_range(start='2022-01-01', periods=100),
    'open': pd.Series(range(100)) + 100,
    'high': pd.Series(range(100)) + 105,
    'low': pd.Series(range(100)) + 95,
    'close': pd.Series(range(100)) + 100,
    'adjusted_close': pd.Series(range(100)) + 100,
    'volume': pd.Series(range(1000, 1100))
})
TSLA['Daily Return'] = TSLA['close'].pct_change() * 100
TSLA['30_day_MA'] = TSLA['close'].rolling(window=30).mean()

# Visualization 1: Tesla Stock Price (Closing Price)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['close'], mode='lines', name='Closing Price'))
fig1.update_layout(title='Tesla Stock Prices (Closing Price)', xaxis_title='Date', yaxis_title='Price (USD)')

# Visualization 2: Tesla Adjusted Stock Price
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['adjusted_close'], mode='lines', name='Adjusted Closing Price', line=dict(color='green')))
fig2.update_layout(title='Tesla Adjusted Stock Prices', xaxis_title='Date', yaxis_title='Price (USD)')

# Visualization 3: Distribution of Daily Returns
fig3 = px.histogram(TSLA, x='Daily Return', nbins=50, title='Distribution of Tesla Daily Returns', color_discrete_sequence=['purple'])
fig3.update_layout(xaxis_title='Daily Return (%)', yaxis_title='Frequency')

# Visualization 4: Correlation Matrix
corr_matrix = TSLA[['open', 'high', 'low', 'close', 'adjusted_close', 'volume']].corr()
fig4 = px.imshow(corr_matrix, text_auto=True, color_continuous_scale='viridis', title='Correlation Matrix of Tesla Stock Data')

# Visualization 5: Closing Price with 30-Day Moving Average
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['close'], mode='lines', name='Closing Price'))
fig5.add_trace(go.Scatter(x=TSLA['date'], y=TSLA['30_day_MA'], mode='lines', name='30-Day Moving Average', line=dict(dash='dash', color='red')))
fig5.update_layout(title='Tesla Stock Prices with 30-Day Moving Average', xaxis_title='Date', yaxis_title='Price (USD)')

# Save figures as HTML for easy conversion
fig1.write_html("figure1.html")
fig2.write_html("figure2.html")
fig3.write_html("figure3.html")
fig4.write_html("figure4.html")
fig5.write_html("figure5.html")

print("Figures saved as HTML. Open them in a browser and print to PDF manually.")


# **Emission and Sustainability Data**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URLs for the EPA and ISO data
epa_url = 'https://www.epa.gov/automotive-trends/highlights-co2-and-fuel-economy-trends'
iso_url = 'https://www.iso.org/standard/43170.html'

# Send GET requests to the URLs
epa_response = requests.get(epa_url)
iso_response = requests.get(iso_url)

# Parse the HTML content
epa_soup = BeautifulSoup(epa_response.content, 'html.parser')
iso_soup = BeautifulSoup(iso_response.content, 'html.parser')

# Extract Tesla emissions data
tesla_emissions = []
for year in range(2015, 2024):
    try:
        tesla_emission = epa_soup.find(f'h3', text=f'Tesla Emissions - {year}').find_next('p').text.strip('Tons')
        tesla_emissions.append((year, float(tesla_emission)))
    except AttributeError:
        print(f"No data found for Tesla Emissions in {year}")

# Extract global emissions regulations data
global_regulations = []
for year in range(2015, 2024):
    try:
        regulation = iso_soup.find(f'h3', text=f'Global Emissions Regulations - {year}').find_next('p').text.strip('%')
        global_regulations.append((year, float(regulation)))
    except AttributeError:
        print(f"No data found for Global Emissions Regulations in {year}")

# Create DataFrames
emissions_df = pd.DataFrame(tesla_emissions, columns=['Year', 'Tesla Emissions'])
regulations_df = pd.DataFrame(global_regulations, columns=['Year', 'Global Emissions Regulations'])

# Merge the data
data_df = pd.merge(emissions_df, regulations_df, on='Year')

# Check for missing values
print(data_df.isnull().sum())

# Save the data to a CSV file
data_df.to_csv('emissions_sustainability_data.csv', index=False)
